# Assignment 2

**Deadline**:  Sunday 12 December, 23.59.

Your notebook should not give errors when executed with `Run All`. Please submit your answers via [Canvas](https://canvas.uva.nl/courses/25150/assignments/248823).

|**Name**|**Student ID**|**Email**|
|:-------|:-------------|:--------|
|        |              |         |


#### Hand in the following:
* Your notebook. **click on `Kernel`, then `Restart & Run All`** before submitting.
* A (printed) pdf version of your notebook.

#### NOTES:
* This is an individual assignment. Teamwork is **not** permitted, including during lectures.
* The assignment is a partial stand-in for a final examination, so the usual rules regarding plagiarism and fraud apply, with all attendant consequences. Code found on the internet or elsewhere is not acceptable as a solution.
* Efficient code is required for full marks.
* Make sure that any function you write has a docstring, and comments where appropriate.
* Before submitting your work, **click on `Kernel`, then `Restart & Run All`** and verify that your notebook produces the desired results and does not error.
* Some questions require you to write code to obtain a numerical result (e.g., an option price). In that case, don't just give the function, but also the result of calling it with the given parameter values (i.e., the numerial value that it returns). If your function uses random numbers, then set the seed to 0 before calling it. This makes it much easier to grade the assignments (at least as long as the answer is correct).
* How to convert your notebook to pdf: The easiest way is probably to use your browser's print functionality: e.g. in Chrome, press the three dots to go to Options, select 'Print...', and then select 'save as pdf' as Destination. I usually set it to 70% zoom such that the output fits the printed pdf pages.


#### Declaration of Originality:
By submitting these answers, I declare that
1. I have read and understood the notes above.
2. These solutions are solely my own work.
3. I have not made these solutions available to any other student.

**Question A**:
Consider an $N$-period model with two assets: a riskless bond with value $B_t=e^{rt}$, and a stock that evolves according to a binomial tree, such that $S_t=S_{t-\Delta t}u$ or $S_t=S_{t-\Delta t}d$ in the good and bad states of the world, respectively. Here $u=1/d=\exp(\sigma\sqrt{\Delta t})$ and $\Delta t= T/N$ as usual. We would like to price a *chooser* option, which has the property that, after $M<N$ periods, the holder can choose whether the option is a European call or put option, both maturing after $N$ periods, and with the same strike price $K$ (See Hull, section 25.7 (ed. 8) or 26.8 (ed. 9, 10, or 11)). Denoting their prices after $i$ periods as $C_{i}$ and $P_{i}$, respectively, the value of the chooser option after $M$ periods is $V_{M}=\max \{C_{M},P_{M}\}\,$. Our starting point is the function `calltree` (or `calltree_numpy`) from Week 5. 

**For each of the following subquestions, your answer should include the function itself, as well as the result of calling it with the following parameter values**: $S_0 = 12$, $K = 15$, $T = 3/12$, $r = 0.02$, $\sigma = 0.25$, $q = 0.01$, $N = 500$, $M=200$.

   1. We want to allow for a dividend yield, $q$. Modify `calltree_numpy` to accept an additional input argument `q`. That is, the function becomes `calltree_numpy(S0, K, T, r, sigma, q, N)`. In the function change the risk-neutral probability to $(e^{(r-q)\Delta t}-d)/(u-d)$.
   2. Building on your function from the previous question write a function `chooser(S0, K, T, r, sigma, q, N, M)` that prices a European chooser option. Along with the call price tree, this function should also compute a put price tree stored in a matrix `P` (of the same size as the call price matrix). Following computation of `C` and `P`, create a $(M+1) \times (M+1)$ matrix `CH`, which stores the tree for the chooser option. At point in time `M`, the last column of this tree will be populated with $V_M=\max \{C_{M},P_{M}\}\,$. Based on the `CH` tree perform the backwards induction to determine the price of the chooser.
   3. Now consider a chooser option for which after $M$ periods, the holder will choose between an *American* call or an *American* put (above we were choosing between a *European* call and a *European* put). Modify the function from above so that before populating the chooser tree `CH` the first loop also determines whether to exercise or keep the respective *American* put and *American* call. Call the function that evaluates the price of this particular derivative `chooser_american(S0, K, T, r, sigma, q, N, M)`.
   4. For the given parameters values ($S_0$, $K$, $T$, $r$, $\sigma$, $q$, $N$) plot the prices of `chooser` and `chooser_american` as a function of $M$. That is, produce a graph that has the prices of the two options on the y-axis and $M$ on the x-axis. Describe and explain (in words) the price convergence.

In [ ]:
def calltree_numpy(S0, K, T, r, sigma, N):
    """European call price based on an N-step binomial tree"""
    deltaT = T / float(N)
    u = np.exp(sigma*np.sqrt(deltaT))
    d = 1.0 / u
    p = (np.exp(r*deltaT)-d) / (u-d)
    piu = np.exp(-r*deltaT) * p
    pid = np.exp(-r*deltaT) * (1-p)
    C = np.zeros((N+1, N+1))
    
    # Stock price
    S = S0 * u**np.arange(N+1) * d**(2*np.arange(N+1)[:, np.newaxis])
    S = np.triu(S)  #keep only the upper triangular part

    # Final payoffs
    C[:, N] = np.maximum(0, S[:, N]-K)  #note: np.maximum in place of max
    
    # Work backwards through the tree
    for j in range(N-1, -1, -1):
        C[:j+1, j] = piu * C[:j+1, j+1] + pid * C[1:j+2, j+1]
        
    return  C[0, 0]

 **Question B**:

 We wish to obtain the no-arbitrage price $X_{0}$ of an index-linked certificate. The derivative has payoff $X_{T}$, defined from the path of an equity index $\{S_{t}\}_{0\leq t\leq T}$ as follows:
 $$
 X_{T}=S_{0}\prod\nolimits_{t=1}^{T}(1+\max (R_{t},R)),\qquad R_{t}=\frac{S_{t}-S_{t-1}}{S_{t-1}},\qquad R=e^{r}-1,
 $$
 where $r\equiv \log (1+R)$ is the continuously compounded interest rate, so that $R$ is the corresponding annually compounded interest rate. Time is measured in years, and $T$ is a positive integer, representing the number of years after which the payoff is received. The idea behind this derivative is that the investor participates in the upside potential of the stock market, but receives a guaranteed minimum return. The value $X_{0}$ of this product cannot be evaluated analytically. 

 We assume a Black-Scholes economy and use the following parameter values: $S_0=9$, $T=10$, $R=0.02$, $\sigma =0.25$, and $q =0.01$. 
     
   1. Using `asianmc_vec` (and `bmsim_vec`) as a starting point, modify the `payoff` of the derivative accordingly. Call the function `certificatemc(S0, R, T, sigma, q, numsim=10000)`, which will return the Monte Carlo estimate of $X_{0}$, along with a 95% confidence interval. **Notice that both the price (at origination) and the CI should be stated explicitly in your answers, based on a random seed of 0**.
   
   2. Test if the confidence interval returned by `certificatemc` has correct coverage, by simulating 1000 independent ones (with `numsim=1000`), and recording in a hit series $I_j$, $j\in\{1, \ldots, 1000\}$, whether an interval contains the true price ($I_j=1$) or not ($I_j=0$). The test can be conducted by regressing $\{I_j-0.95\}_{j=1}^{1000}$ on an intercept and testing whether that is zero. The 'true' price should be obtained from `certificatemc` with `numsim=100000`.